In [1]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import re

In [2]:
# Define a request header (to prevent anti-scraping)
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

 ### Title, Subjects, Authors, and abstract

In [3]:
url = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
response = requests.get(url, headers = headers)
print(response.status_code)
page = response.text
soup = BeautifulSoup(page, "html.parser")
# print(page)
indexs = soup.find_all("span", attrs={"class" : "list-identifier"})
titles = soup.findAll("div", attrs={"class" : "list-title mathjax"})
subjects = soup.findAll("div", attrs={"class" : "list-subjects"})

#print(page)

200


### title

In [4]:
# print(titles[0].prettify())
# print("#########")
pattern = re.compile(r'</span>(.*?)</div>', re.DOTALL)  
title_list = []

for title in titles:
    match = pattern.search(title.prettify())
    if match:
        title_list.append(match.group(1).strip())

### index

In [5]:
# print(str(indexs[0]))
# print("#########")
pattern = re.compile(r'<span class="list-identifier"><a href="(.*?)" title="Abstract">arXiv:(.*?)</a>', re.DOTALL)  
index_list = []

for index in indexs:
    match = pattern.search(str(index))
    if match:
        index_list.append(match.group(1).strip())
# index_list

### subject

In [6]:
# print(str(subjects[0]))
# print("#")
# print(str(subjects[1]))
pattern1 = re.compile(r'<span class="primary-subject">(.*?)</span>', re.DOTALL)  
pattern2 = re.compile(r'</span>; (.*?)</div>', re.DOTALL)  

subject_list = []

for subject in subjects:
    match1 = pattern1.search(str(subject))
    match2 = pattern2.search(str(subject))
    if match2:
        subject_list.append(match1.group(1).strip()+"; "+ match2.group(1).strip())
    else:
        subject_list.append(match1.group(1).strip())

# subject_list

### abs & author

In [7]:
url = "https://arxiv.org"
pattern = re.compile(r'<meta content=(.*?) property="og:description"/>')
abs_list = []
author_list = []
i = 0
j = 0
for index in index_list:
    urli = f"{url}{index}"
    print(f'i = {i}, url = {urli}')
    response = requests.get(urli, headers = headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    abstract = soup.findAll("meta", attrs={"property" : "og:description"})
    authors_abs = soup.find_all("meta", attrs={"name" : "citation_author"})
    
    # abs
    abs_str = str(abstract)
    match = pattern.search(abs_str)
    if match:
        abs_list.append(match.group(1).strip())
        print(f'catch')
    
    # authors
    authors = '"'

    for author in authors_abs:
        author_str = str(author)
        authors += author_str[15:-26]+"; "
        
    authors = authors[:-2]+'"'
    author_list.append(authors)
    
    i += 1
    time.sleep(15)
print(abs_list[99])
    


i = 0, url = https://arxiv.org/abs/2403.02338
catch
i = 1, url = https://arxiv.org/abs/2403.02336
catch
i = 2, url = https://arxiv.org/abs/2403.02334
catch
i = 3, url = https://arxiv.org/abs/2403.02333
catch
i = 4, url = https://arxiv.org/abs/2403.02332


KeyboardInterrupt: 

In [ ]:
data = {'Title' : title_list, 'Authors' : author_list, 'Subject' : subject_list, 'Abstruct' : abs_list}
df = pd.DataFrame(data)

df.to_csv('output.csv', index = False)
